<a href="https://colab.research.google.com/github/antoniod621/CRUD-App-Students/blob/main/2_group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install mrjob
!pip install mrjob

     |████████████████████████████████| 439 kB 4.6 MB/s 


In [5]:
from google.colab import files

# upload dataset
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving nashville-tweets-2019-01-28 to nashville-tweets-2019-01-28
User uploaded file "nashville-tweets-2019-01-28" with length 25039251 bytes


# Using multi steps

A MR job has multiple steps. Read about reducer and steps in https://buildmedia.readthedocs.org/media/pdf/mrjob/latest/mrjob.pdf

A “step” consists of a mapper, a combiner, and a reducer. All of those are optional, though you must have at least one.
So you could have a step that’s just a mapper, or just a combiner and a reducer.
When you only have one step, all you have to do is write methods called mapper(), combiner(), and
reducer()

In [50]:
%%file 2_group.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import json

'''


See http://mike.teczno.com/notes/streaming-data-from-twitter.html for parsing info.
Get the screen name by accessing tweet['user']['screen_name']

Output:
number_of_most_tweets    username

(note that MRJob automatically adds a tab for you)
'''
# read the 
class GroupMaxTweets(MRJob):
    # The _ means the field does not matter.;
    def mapper(self, _, line):
        try:
            # do something
            tweet = json.loads(line)
            yield tweet['user']['screen_name'], 1
        except:
            pass

    def reducer(self, key, counts):
        # yield something -- hint you can yield a tuple
        yield None, (sum(counts), key)
   
    def reducer_max(self, key, counts):
        # yield something
        yield max(counts)
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper, reducer=self.reducer),
            MRStep(reducer=self.reducer_max)
        ]


if __name__ == '__main__':
    GroupMaxTweets.run()

Overwriting 2_group.py


## Step 1. Run MR program locally(Colab)

In [51]:
%%timeit
!python 2_group.py -r local nashville-tweets-2019-01-28 > 2_group.out

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/2_group.root.20220331.162721.730247
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/2_group.root.20220331.162721.730247/output
Streaming final output from /tmp/2_group.root.20220331.162721.730247/output...
Removing temp directory /tmp/2_group.root.20220331.162721.730247...
No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/2_group.root.20220331.162724.862445
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/2_group.root.20220331.162724.862445/output
Streaming final output from /tmp/2_group.root.20220331.162724.862445/output...
Removing temp directory /tmp/2_group.root.20220331.162724.862445...
No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/2_group.root.20220331.162728.009059
Running step 1 o

In [52]:
!cat 2_group.out

88	"rpsabo"


## Step 2. Run MR program on AWS EMR

### Please copy nashville-tweets-2019-01-28 into your S3 bucket.

In [ ]:
%%timeit
!python 2_group.py -r emr s3://<s3 url of the Tweet dataset> --cloud-tmp-dir=s3://<s3 url of tmp directory> --cluster-id=<cluster ID> --conf-path <mrjob.conf file path in colab> > 2_group.out

In [53]:
#test the code run

f = open('2_group.out')
lines = f.readlines()
f.close()

assert lines[0][:-1] in ['88\t"rpsabo"']
print("test passed")

test passed
